In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import re
import os

class ValidadorDeContrasenas:
    def validar(self, contrasena, confirmacion):
        errores = []


        if len(contrasena) < 8:
            errores.append("La contraseña debe tener al menos 8 caracteres.")

        if " " in contrasena:
            errores.append("La contraseña no debe contener espacios.")

        if not re.search(r'[a-z]', contrasena):
            errores.append("Debe tener al menos una letra minúscula.")

        if not re.search(r'[A-Z]', contrasena):
            errores.append("Debe tener al menos una letra mayúscula.")

        if not re.search(r'\d', contrasena):
            errores.append("Debe tener al menos un número.")

        if not re.search(r'[!@#$%^&*(),.?":{}|<>]', contrasena):
            errores.append("Debe tener al menos un carácter especial.")

        if contrasena != confirmacion:
            errores.append("Las contraseñas no coinciden.")

        return errores

class LectorDeArchivos:
    def leer(self, ruta):
        lineas = []
        try:
            archivo = open(ruta, "r", encoding="utf-8")
            linea = archivo.readline()
            while linea:
                lineas.append(linea)
                linea = archivo.readline()
            archivo.close()
            return "".join(lineas)
        except Exception as e:
            raise IOError("Error al leer el archivo: " + str(e))


class EscritorDeArchivos:
    def escribir(self, ruta, contenido):
        try:
            archivo = open(ruta, "w", encoding="utf-8")
            archivo.write(contenido)
            archivo.close()
            return True
        except Exception as e:
            raise IOError("No se pudo escribir el archivo: " + str(e))

class Aplicacion:
    def __init__(self, root):
        self.root = root
        self.root.title("App de Archivos y Contraseñas (Estilo Estudiante)")
        self.root.geometry("800x600")

        self.validador = ValidadorDeContrasenas()
        self.lector = LectorDeArchivos()
        self.escritor = EscritorDeArchivos()

        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill="both", expand=True)

        self.crear_pestana_contrasena()
        self.crear_pestana_lector()
        self.crear_pestana_escritor()

    def crear_pestana_contrasena(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Validar Contraseña")

        ttk.Label(frame, text="Contraseña:").pack(pady=5)
        self.entry_contrasena = ttk.Entry(frame, show="*")
        self.entry_contrasena.pack()

        ttk.Label(frame, text="Confirmar Contraseña:").pack(pady=5)
        self.entry_confirmacion = ttk.Entry(frame, show="*")
        self.entry_confirmacion.pack()

        self.resultado_validacion = ttk.Label(frame, text="", foreground="red")
        self.resultado_validacion.pack(pady=10)

        btn_validar = ttk.Button(frame, text="Validar", command=self.validar_contrasena)
        btn_validar.pack(pady=5)

    def validar_contrasena(self):
        contrasena = self.entry_contrasena.get()
        confirmacion = self.entry_confirmacion.get()

        errores = self.validador.validar(contrasena, confirmacion)

        if len(errores) == 0:
            self.resultado_validacion.config(text="¡Contraseña válida!", foreground="green")
        else:
            mensaje = "\n".join(errores)
            self.resultado_validacion.config(text=mensaje, foreground="red")

    def crear_pestana_lector(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Leer Archivo")

        self.entry_ruta_lector = ttk.Entry(frame, width=60)
        self.entry_ruta_lector.pack(pady=5)

        btn_buscar = ttk.Button(frame, text="Buscar archivo", command=self.seleccionar_archivo)
        btn_buscar.pack(pady=5)

        btn_leer = ttk.Button(frame, text="Leer contenido", command=self.mostrar_archivo)
        btn_leer.pack(pady=5)

        btn_mayus = ttk.Button(frame, text="Convertir a MAYÚSCULAS", command=self.convertir_a_mayus)
        btn_mayus.pack(pady=5)

        btn_limpiar = ttk.Button(frame, text="Limpiar", command=self.limpiar_texto_lector)
        btn_limpiar.pack(pady=5)

        self.texto_lector = tk.Text(frame, height=20)
        self.texto_lector.pack(padx=10, pady=10, fill="both", expand=True)

    def seleccionar_archivo(self):
        ruta = filedialog.askopenfilename()
        if ruta:
            self.entry_ruta_lector.delete(0, tk.END)
            self.entry_ruta_lector.insert(0, ruta)

    def mostrar_archivo(self):
        ruta = self.entry_ruta_lector.get()
        if not os.path.exists(ruta):
            messagebox.showerror("Error", "El archivo no existe.")
            return
        try:
            contenido = self.lector.leer(ruta)
            self.texto_lector.delete("1.0", tk.END)
            self.texto_lector.insert(tk.END, contenido)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def convertir_a_mayus(self):
        texto = self.texto_lector.get("1.0", tk.END)
        self.texto_lector.delete("1.0", tk.END)
        self.texto_lector.insert(tk.END, texto.upper())

    def limpiar_texto_lector(self):
        self.texto_lector.delete("1.0", tk.END)

    def crear_pestana_escritor(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Escribir Archivo")

        self.entry_ruta_escritor = ttk.Entry(frame, width=60)
        self.entry_ruta_escritor.pack(pady=5)

        btn_guardar_como = ttk.Button(frame, text="Guardar como...", command=self.seleccionar_guardado)
        btn_guardar_como.pack(pady=5)

        btn_guardar = ttk.Button(frame, text="Guardar", command=self.guardar_archivo)
        btn_guardar.pack(pady=5)

        self.texto_escritor = tk.Text(frame, height=20)
        self.texto_escritor.pack(padx=10, pady=10, fill="both", expand=True)

    def seleccionar_guardado(self):
        ruta = filedialog.asksaveasfilename(defaultextension=".txt")
        if ruta:
            self.entry_ruta_escritor.delete(0, tk.END)
            self.entry_ruta_escritor.insert(0, ruta)

    def guardar_archivo(self):
        ruta = self.entry_ruta_escritor.get()
        contenido = self.texto_escritor.get("1.0", tk.END)

        if ruta.strip() == "":
            messagebox.showerror("Error", "Debes indicar una ruta para guardar.")
            return

        if contenido.strip() == "":
            messagebox.showerror("Error", "El contenido está vacío.")
            return

        try:
            self.escritor.escribir(ruta, contenido)
            messagebox.showinfo("Éxito", "Archivo guardado correctamente.")
        except Exception as e:
            messagebox.showerror("Error", str(e))


if __name__ == "__main__":
    ventana = tk.Tk()
    app = Aplicacion(ventana)
    ventana.mainloop()
